<a href="https://colab.research.google.com/github/yymin1022/MCC_Assignments/blob/main/Project_4/prob2/prob2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
%%writefile thrust_ex.cu
#include <thrust/host_vector.h>
#include <thrust/device_vector.h>
#include <thrust/transform_reduce.h>
#include <thrust/functional.h>
#include <thrust/iterator/counting_iterator.h>
#include <iostream>
#include <cuda_runtime.h>

// Function f(x) = 4.0 / (1 + x*x)
struct functor {
    __host__ __device__
    float operator()(const float& x) const {
        return 4.0 / (1.0 + x * x);
    }
};

// Function to be applied in transform_reduce
struct integral_functor {
    float dx;
    functor f;

    integral_functor(float _dx) : dx(_dx) {}

    __host__ __device__
    float operator()(const int& i) const {
        float x = (i + 0.5f) * dx;
        return f(x) * dx;
    }
};

int main() {
    const int N = 1000000000;
    const float dx = 1.0f / N;

    thrust::counting_iterator<int> begin(0);
    thrust::counting_iterator<int> end(N);

    cudaEvent_t time_start, time_stop;
    cudaEventCreate(&time_start);
    cudaEventCreate(&time_stop);

    cudaEventRecord(time_start, 0);

    float integral = thrust::transform_reduce(
        begin, end,
        integral_functor(dx),
        0.0f,
        thrust::plus<float>()
    );

    cudaEventRecord(time_stop, 0);
    cudaEventSynchronize(time_stop);

    float time_elapsed = 0;
    cudaEventElapsedTime(&time_elapsed, time_start, time_stop);

    std::cout << "Approximate integral: " << integral << std::endl;
    std::cout << "Elapsed time: " << time_elapsed << " ms" << std::endl;

    return 0;
}


Overwriting thrust_ex.cu


In [4]:
!nvcc thrust_ex.cu -o thrust_ex
!./thrust_ex

Approximate integral: 3.14159
Elapsed time: 244.037 ms


In [5]:
%%writefile omp_pi_one.c
#include <omp.h>
#include <stdio.h>

long num_steps = 1000000000;
double step;

void main ()
{
	long i; double x, pi, sum = 0.0;
	double start_time, end_time;

	omp_set_num_threads(1);
	start_time = omp_get_wtime();
	step = 1.0/(double) num_steps;
	for (i=0;i< num_steps; i++){
		x = (i+0.5)*step;
		sum = sum + 4.0/(1.0+x*x);
	}
	pi = step * sum;
	end_time = omp_get_wtime();
	double timeDiff = end_time - start_time;
        printf("Execution Time : %.10lfsec\n", timeDiff);

	printf("pi=%.10lf\n",pi);
}

Writing omp_pi_one.c


In [6]:
!gcc -fopenmp omp_pi_one.c -o omp_pi_one
!./omp_pi_one

Execution Time : 3.3276022480sec
pi=3.1415926536
